# Задание 3

В базе данных вашей компании присутствуют следующие таблицы:

* city
* client
* promotion
* purchase

### Вам нужно написать запрос, чтобы получить такую таблицу

* **purchase_date** – дата покупки
* **purchase_id** – id покупки
* **client_id** – id покупателя
* **client_age** – возраст покупателя
* **client_registration_age** – как долго человек пользуется вашими магазинами
* **client_category** – содержит **new** или **old**, если это первая или последующая покупка соответственно
* **promotion_name** – акция
* **category_name** – категория
* **partner_name** – партнёр
* **client_city** – город, где находится покупатель
* **city** – город
* **revenue** – сумма выручки
* **quantity** – число проданных единиц

*При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020*

In [1]:
import pandas as pd
import pandahouse as ph

### Подключение к СУБД

Прописываем учетные данные для подключения к серверу:

In [2]:
connection = dict(database = 'project_variant_2',
                  host = 'https://clickhouse.lab.karpov.courses',
                  user = 'student',
                  password = 'dpo_python_2020')

Определяем SQL-запрос для вывода итоговой таблицы:

In [3]:
query = '''
SELECT
    d.purchase_date as purchase_date,
    d.purchase_id as purchase_id,
    d.client_id as client_id,
    dateDiff(YEAR, b.birth_date, NOW()) as client_age,
    dateDiff(YEAR, b.birth_date, b.registration) AS client_registration_age,
    if(d.purchase_id > f.min_purchase_id, 'old', 'new') as client_category,
    c.promotion_name as promotion_name,
    c.category_name as category_name,
    c.partner_name as partner_name,
    a.client_city as client_city,
    a.city as city,
    CAST(d.quantity AS Float32) * CAST(d.price AS Float32) as revenue,
    CAST(d.quantity AS Float32) as quantity
FROM project_variant_2.purchase as d
JOIN 
    (
    SELECT 
        birth_date,
        registration,
        client_id
    FROM 
        project_variant_2.client
    ) as b
    on d.client_id = b.client_id
JOIN
    (
    SELECT
        promotion_name,
        category_name,
        partner_name,
        promotion_id
    FROM
        project_variant_2.promotion
    ) as c
    on d.promotion_id = c.promotion_id
LEFT SEMI JOIN
    (
    SELECT
        client_city,
        city,
        city_id
    FROM project_variant_2.city
    ) as a
    on d.city_id = a.city_id
JOIN
    (
    SELECT
        min(purchase_id) as min_purchase_id,
        client_id
    FROM 
        project_variant_2.purchase
    GROUP BY
        client_id
    ) as f
    on f.client_id=d.client_id
WHERE 
        d.status = 1
    AND d.purchase_date >= '2020-05-01'
    AND d.purchase_date < '2020-08-01'
ORDER BY client_id
'''

Создаем необходимый датафрейм, обращаясь к **СУБД** через фреймворк **pandahouse**:

In [4]:
df = ph.read_clickhouse(query, connection=connection)

Проверим итоговый датафрейм:

In [5]:
df.head()

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-07-26,918,1,26,25,new,campaign_4,Прочее,KarpovCourses,Санкт-Петербург,Минск,615.0,3.0
1,2020-07-20,3002,1,26,25,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Минск,Новгород,660.0,3.0
2,2020-07-24,3626,1,26,25,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Новгород,Ярославль,2280.0,8.0
3,2020-06-08,6111,2,23,20,old,campaign_3,Овощи,Всем партнёрам партнёр,Санкт-Петербург,Минск,360.0,1.0
4,2020-05-21,6702,2,23,20,old,campaign_4,Прочее,KarpovCourses,Санкт-Петербург,Минск,465.0,3.0


In [6]:
df.shape

(2695, 13)

In [7]:
df.dtypes

purchase_date              datetime64[ns]
purchase_id                         int32
client_id                           int32
client_age                          int64
client_registration_age             int64
client_category                    object
promotion_name                     object
category_name                      object
partner_name                       object
client_city                        object
city                               object
revenue                           float64
quantity                          float32
dtype: object

In [8]:
df.isna().sum()

purchase_date              0
purchase_id                0
client_id                  0
client_age                 0
client_registration_age    0
client_category            0
promotion_name             0
category_name              0
partner_name               0
client_city                0
city                       0
revenue                    0
quantity                   0
dtype: int64

In [11]:
min(df.purchase_date)

Timestamp('2020-05-01 00:00:00')

In [12]:
max(df.purchase_date)

Timestamp('2020-07-31 00:00:00')

* Создание датафрейма выполнено успешно, датафрейм состоит только из необходимых нам колонок;
* Пропущенные значения отсутствуют;
* Типы данных соответствуют значениям в колонках;
* Временной интервал соответствует запрашиваемому (от 01-05-2020 по 31-07-2020 включительно).

### Запись полученной таблицы в итоговый csv-файл

In [9]:
df.to_csv('sql_df.csv', index=False)